<a href="https://colab.research.google.com/github/usshaa/A-complete-guide-to-DL/blob/master/MobileNetV2_from_TensorFlow_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [2]:
# Load a small dataset (TFDS will download it automatically)
dataset_name = "cats_vs_dogs"
(train_ds, val_ds), ds_info = tfds.load(
    dataset_name,
    split=["train[:80%]", "train[80%:]"],
    with_info=True,
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.8POHUP_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [3]:
# Preprocessing function
IMG_SIZE = 224

In [4]:
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0  # Normalize
    return image, label

In [5]:
train_ds = train_ds.map(preprocess).shuffle(500).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

In [6]:
# Load MobileNetV2 from TensorFlow Hub (feature extractor, no top layer)
mobilenet_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [7]:
feature_extractor_layer = hub.KerasLayer(mobilenet_url,
                                         input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                         trainable=False)

In [12]:
# Build the model
model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(2, activation='softmax')  # 2 classes: cats & dogs
])

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7a06d0182250> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=1)

In [ ]:
# Plot training history
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Test with a single image from validation set
for image, label in val_ds.take(1):
    pred = model.predict(image)
    print("Predicted:", tf.argmax(pred[0]).numpy(), "Actual:", label[0].numpy())
    plt.imshow(image[0])
    plt.show()